In [1]:
import numpy as np
import pandas as pd
from tensorflow.keras.utils import to_categorical
import matplotlib.pyplot as plt
import seaborn as sns


def load_file(filepath):
    dataframe = pd.read_csv(filepath, header=None, delim_whitespace=True)
    return dataframe.values

# load a list of files and return as a 3d numpy array
def load_group(filenames, prefix=''):
    loaded = list()
    for name in filenames:
        data = load_file(prefix + name)
        loaded.append(data)
    # stack group so that features are the 3rd dimension
    loaded = np.dstack(loaded)
    return loaded

# load a dataset group, such as train or test
def load_dataset_group(group, prefix=''):
    filepath = prefix + group + '/Inertial Signals/'
    # load all 9 files as a single array
    filenames = list()
    # total acceleration
    filenames += ['total_acc_x_'+group+'.txt', 'total_acc_y_'+group+'.txt', 'total_acc_z_'+group+'.txt']
    # body acceleration
    filenames += ['body_acc_x_'+group+'.txt', 'body_acc_y_'+group+'.txt', 'body_acc_z_'+group+'.txt']
    # body gyroscope
    filenames += ['body_gyro_x_'+group+'.txt', 'body_gyro_y_'+group+'.txt', 'body_gyro_z_'+group+'.txt']
    # load input data
    X = load_group(filenames, filepath)
    # load class output
    y = load_file(prefix + group + '/y_'+group+'.txt')
    return X, y


# load the dataset, returns train and test X and y elements
def load_dataset(prefix=''):
    # load all train
    trainX, trainy = load_dataset_group('train', prefix + './HAR/UCI/')
    # load all test
    testX, testy = load_dataset_group('test', prefix + './HAR/UCI/')
    
    #zero-offset class values
    trainy = trainy - 1
    testy = testy - 1
    #one hot encode y
    trainy_one_hot = to_categorical(trainy)
    testy_one_hot = to_categorical(testy)
    print(trainX.shape, trainy.shape, trainy_one_hot.shape, testX.shape, testy.shape, testy_one_hot.shape)
    return trainX, trainy, trainy_one_hot, testX, testy, testy_one_hot

In [2]:
X_train, y_train, y_train_one_hot, X_test, y_test, y_test_one_hot = load_dataset()

(7352, 128, 9) (7352, 1) (7352, 6) (2947, 128, 9) (2947, 1) (2947, 6)


In [3]:
n_timesteps, n_features, n_outputs = X_train.shape[1], X_train.shape[2], y_train_one_hot.shape[1]

In [4]:
n_timesteps, n_features, n_outputs

(128, 9, 6)

In [5]:
from tensorflow.keras.layers import Input, Bidirectional, LSTM, Dense
from tensorflow.keras.models import Model
from tensorflow.keras.utils import to_categorical

# BiLSTM 모델 생성
# 입력 시퀀스는 X_train의 길이만큼 설정
input_layer = Input(shape=(X_train.shape[1], X_train.shape[2]))
bi_lstm = Bidirectional(LSTM(64))(input_layer)
output_layer = Dense(y_train_one_hot.shape[1], activation='softmax')(bi_lstm)

BiLSTM_model = Model(inputs=input_layer, outputs=output_layer)
BiLSTM_model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])

# 모델 훈련
BiLSTM_model.fit(X_train, y_train_one_hot, epochs=10, batch_size=32, validation_data=(X_test, y_test_one_hot))

# 모델 평가
loss, accuracy = BiLSTM_model.evaluate(X_test, y_test_one_hot)
print(f"Test Loss: {loss}, Test Accuracy: {accuracy}")

Epoch 1/10
230/230 [==============================] - 12s 41ms/step - loss: 1.0417 - accuracy: 0.5721 - val_loss: 0.9145 - val_accuracy: 0.6111
Epoch 2/10
230/230 [==============================] - 9s 38ms/step - loss: 0.6493 - accuracy: 0.7482 - val_loss: 0.5820 - val_accuracy: 0.7900
Epoch 3/10
230/230 [==============================] - 9s 39ms/step - loss: 0.5047 - accuracy: 0.8130 - val_loss: 0.5462 - val_accuracy: 0.7754
Epoch 4/10
230/230 [==============================] - 9s 39ms/step - loss: 0.3433 - accuracy: 0.8749 - val_loss: 0.4490 - val_accuracy: 0.8351
Epoch 5/10
230/230 [==============================] - 9s 39ms/step - loss: 0.2426 - accuracy: 0.9172 - val_loss: 0.4245 - val_accuracy: 0.8561
Epoch 6/10
230/230 [==============================] - 9s 39ms/step - loss: 0.2373 - accuracy: 0.9196 - val_loss: 0.3321 - val_accuracy: 0.8789
Epoch 7/10
230/230 [==============================] - 9s 40ms/step - loss: 0.2214 - accuracy: 0.9204 - val_loss: 0.4222 - val_accuracy: 0.863

In [6]:
import numpy as np
from tensorflow.keras.layers import Input, GRU, Dense
from tensorflow.keras.models import Model
from tensorflow.keras.utils import to_categorical

# GRU 모델 생성
input_layer = Input(shape=(X_train.shape[1], X_train.shape[2]))
gru_layer = GRU(64)(input_layer)
output_layer = Dense(y_train_one_hot.shape[1], activation='softmax')(gru_layer)

GRU_model = Model(inputs=input_layer, outputs=output_layer)
GRU_model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])

# 모델 훈련
GRU_model.fit(X_train, y_train_one_hot, epochs=10, batch_size=32, validation_data=(X_test, y_test_one_hot))

# 모델 평가
loss, accuracy = GRU_model.evaluate(X_test, y_test_one_hot)
print(f"Test Loss: {loss}, Test Accuracy: {accuracy}")


Epoch 1/10
230/230 [==============================] - 9s 34ms/step - loss: 1.1670 - accuracy: 0.5135 - val_loss: 1.2373 - val_accuracy: 0.5226
Epoch 2/10
230/230 [==============================] - 7s 32ms/step - loss: 0.7476 - accuracy: 0.6873 - val_loss: 0.7382 - val_accuracy: 0.7272
Epoch 3/10
230/230 [==============================] - 8s 33ms/step - loss: 0.4211 - accuracy: 0.8541 - val_loss: 0.4705 - val_accuracy: 0.8164
Epoch 4/10
230/230 [==============================] - 8s 34ms/step - loss: 0.1819 - accuracy: 0.9317 - val_loss: 0.3373 - val_accuracy: 0.8829
Epoch 5/10
230/230 [==============================] - 8s 34ms/step - loss: 0.1568 - accuracy: 0.9395 - val_loss: 0.3287 - val_accuracy: 0.8992
Epoch 6/10
230/230 [==============================] - 8s 36ms/step - loss: 0.1382 - accuracy: 0.9421 - val_loss: 0.3330 - val_accuracy: 0.8931
Epoch 7/10
230/230 [==============================] - 8s 36ms/step - loss: 0.1279 - accuracy: 0.9501 - val_loss: 0.3188 - val_accuracy: 0.8982

In [7]:
import numpy as np
from tensorflow.keras.layers import Input, Bidirectional, GRU, Dense
from tensorflow.keras.models import Model
from tensorflow.keras.utils import to_categorical

# BiGRU 모델 생성
input_layer = Input(shape=(X_train.shape[1], X_train.shape[2]))
bigru_layer = Bidirectional(GRU(64))(input_layer)
output_layer = Dense(y_train_one_hot.shape[1], activation='softmax')(bigru_layer)

BiGRU_model = Model(inputs=input_layer, outputs=output_layer)
BiGRU_model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])

# 모델 훈련
BiGRU_model.fit(X_train, y_train_one_hot, epochs=10, batch_size=32, validation_data=(X_test, y_test_one_hot))

# 모델 평가
loss, accuracy = BiGRU_model.evaluate(X_test, y_test_one_hot)
print(f"Test Loss: {loss}, Test Accuracy: {accuracy}")


Epoch 1/10
230/230 [==============================] - 13s 43ms/step - loss: 1.1143 - accuracy: 0.5426 - val_loss: 1.0131 - val_accuracy: 0.5765
Epoch 2/10
230/230 [==============================] - 9s 37ms/step - loss: 0.5079 - accuracy: 0.7982 - val_loss: 0.5431 - val_accuracy: 0.8073
Epoch 3/10
230/230 [==============================] - 9s 37ms/step - loss: 0.2751 - accuracy: 0.9027 - val_loss: 0.3571 - val_accuracy: 0.8894
Epoch 4/10
230/230 [==============================] - 8s 37ms/step - loss: 0.1609 - accuracy: 0.9389 - val_loss: 0.4167 - val_accuracy: 0.8761
Epoch 5/10
230/230 [==============================] - 9s 37ms/step - loss: 0.1477 - accuracy: 0.9415 - val_loss: 0.3303 - val_accuracy: 0.8870
Epoch 6/10
230/230 [==============================] - 9s 37ms/step - loss: 0.1377 - accuracy: 0.9441 - val_loss: 0.3115 - val_accuracy: 0.8867
Epoch 7/10
230/230 [==============================] - 8s 37ms/step - loss: 0.1316 - accuracy: 0.9455 - val_loss: 0.3069 - val_accuracy: 0.903